# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 5: Model Evaluation and Regularization


In this assignment, we will investigate evaluation methods and regularization, which can be used to control the behavior of linear models. Most of the code here will be copied or refactored from previous assignments. You are encouraged to copy **your code only** from previous assignments.

In [144]:
require './assignment_lib'

false

# Question 1.1 (1 Point)

Reuse **YOUR** implementation of ```StochasticGradientDescent``` from a previous assignment.

In [145]:
# BEGIN YOUR CODE




class StochasticGradientDescent
  attr_reader :weights
  attr_reader :objective
  def initialize obj, w_0, lr 
    @objective = obj
    @weights = w_0
    @n = 1.0
    @lr = lr
  end
  def update x
    # BEGIN YOUR CODE
    
    lr  = @lr.to_f / Math.sqrt(@n)
    @n +=1

    if x.empty?
    else
      feature = x[0]["features"].keys
    end

    if (@weights.keys).empty? and  x.length>0
      i=0
      while i<feature.length do
        @weights.store(feature[i],@weights[0])
        i+=1
      end
      
    end

    @weights.delete(0)

    i = 0
    gradient = @objective.grad x,@weights

    while i < @weights.length do
      if @weights[@weights.keys[i]].is_a?Numeric and gradient[@weights.keys[i]].is_a?Numeric
        @weights[@weights.keys[i]]  = @weights[@weights.keys[i]] - lr*gradient[@weights.keys[i]]        
      end
      i+=1
    end    

   @objective.adjust @weights


    #END YOUR CODE
  end
end
#END YOUR CODE

:update

In [146]:
### Hidden Test ###

## Question 1.2 (1 point)

Copy **YOUR** implementation of the ```dot``` product and ```norm``` functions from a previous assignment into the following cell. Please copy the whole function, not just the parts within the comments.

In [147]:
# BEGIN YOUR CODE



#Implement the error function given a weight vector, w
def dot x, w
  # BEGIN YOUR CODE
    x_name = x.keys
    w_name = w.keys

    i = 0
    dot = 0.0
    if x_name.length == 0 
      dot =  0.0
    else 
        while i<x_name.length do

          if w_name.include?x_name[i]
            index = w_name.index { |x| [x_name[i]].include?(x) }
            if x[x_name[i]].is_a? Numeric  and w[w_name[index]].is_a? Numeric
              dot += x[x_name[i]]*w[w_name[index]]
            else 
              dot +=0.0
            end
          else 
            dot += 0.0
          end
          i+=1
        end  
    end
  return dot
  #END YOUR CODE
end




def norm w
  # BEGIN YOUR CODE
  return Math.sqrt(dot w, w)
  #END YOUR CODE
end

#END YOUR CODE

:norm

In [148]:
### Hidden test ###


## Question 1.3 (1 Point)

Refactor **YOUR** $z$-score normalization method from a previous assignment, where it might have been called  ```create_zspambase```. It should be general enough to normalize any dataset. Normalize features in the ```features``` key of the dataset and not all features available in the examples.

Note: Watch out for zero-stdev features.

In [149]:
## Add mean and stdev here

# BEGIN YOUR CODE
def mean x
  # BEGIN YOUR CODE
  x.inject(0.0){|sum,i| sum + i }/x.length()  
  #END YOUR CODE
end


def stdev x
  # BEGIN YOUR CODE
  mu = mean x
  su = x.inject(0.0){|sum,i| sum + (i-mu)**2 }/(x.length()-1)
  Math.sqrt(su) 
end  


def normalization dataset

    dat = dataset["data"]  
    feature = dataset["features"]
    data =   Array.new(feature.length) { Array.new(dat.length){0}}

    i = 0

    while i<dat.length do
      j = 0  
      while j<feature.length do
        data[j][i]=dat[i]["features"][feature[j]]
        j+=1
      end

      i+=1
    end

    means = Hash.new
    stdev = Hash.new

    j = 0  

    while j<feature.length do
        m = mean data[j].reject { |e| e.to_s.empty? }
        s = stdev data[j].reject { |e| e.to_s.empty? }
        means.store(feature[j],m)
        stdev.store(feature[j],s)
        j+=1
    end  
  
return [means,stdev]

end
  #END YOUR CODE



#END YOUR CODE

:normalization

In [150]:
def z_normalize dataset
  zdataset = dataset.clone
  zdataset["data"] = dataset["data"].collect do |r|
    u = r.clone
    u["features"] = r["features"].clone
    u
  end

  # BEGIN YOUR CODE
  

  i = 0

  feature = zdataset["features"]
  norm = normalization(dataset)
  means = norm[0]
  stdev = norm[1]
  while i<  zdataset["data"].length do
    
    j = 0  
    name = zdataset["data"][i]["features"].keys    
    while j<feature.length do
      if name.include?feature[j]
        
         if stdev[feature[j]] == 0
             zdataset["data"][i]["features"][feature[j]] =zdataset["data"][i]["features"][feature[j]] 
         else
         zdataset["data"][i]["features"][feature[j]] = (zdataset["data"][i]["features"][feature[j]]-means[feature[j]])/stdev[feature[j]]
         end
      end
      j+=1
    end

    i+=1
  end  
  
  

  
  #END YOUR CODE
  return zdataset
end

:z_normalize

In [151]:
### TEST ###
def test_9141ec()
  spambase = read_sparse_data_from_csv "spambase"
  zspambase = z_normalize spambase

  assert_in_delta 0.27, spambase["data"].first["features"]["word_freq_our"], 1e-5
  assert_in_delta -0.628106690674003, zspambase["data"].first["features"]["word_freq_our"], 1e-5

  assert_in_delta 607.0, spambase["data"].first["features"]["capital_run_length_total"], 1e-5
  assert_in_delta 0.53386, zspambase["data"].first["features"]["capital_run_length_total"], 1e-5
end

test_9141ec()

## Question 2.1 (5 Points)

Change your ```LinearRegression``` implementation from to implement regularization. The new implementation requires a value for $\lambda$. The regularization objective function for linear regression in a mini-batch is as follows:

# $L(w,X) = \frac{\lambda}{2} \left\lVert w \right\rVert ^ 2 + \frac{1}{n} \sum_{i} \frac{1}{2} \left(f(w,x_i) - y_i\right) ^ 2$

where ```reg_param``` corresponds to $\lambda$ in the formula above.

Note that there is no $\frac{1}{n}$ in front of the regularizer penalty. The ```predict``` and ```adjust``` methods have been provided for you. 

Hint: Use ```dot``` and ```norm``` as needed.

In [152]:
class LinearRegressionModelL2
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x = row["features"]   
    yhat = dot(w, x)
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
  
  def func examples, w
    # BEGIN YOUR CODE
    
    i = 0
    sum = 0.0


    adjust w


    while i< examples.length do
      yhat = predict(examples[i], w)

      ylabel = examples[i]["label"]
      sum +=(yhat -ylabel)**2
      i+=1
    end
 return sum*0.5/examples.length + @reg_param*( norm(w) **2)*0.5
    
    
    #END YOUR CODE
  end
end

:func

In [153]:
### TEST ###
def test_d0ce9f()
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  e1 = 0.19845
  assert_in_delta e1, m.func(examples[0,1], w), 1e-2, "1"
  
  e2 = 2.42
  assert_in_delta e2, m.func(examples[1,1], w), 1e-2, "2"    
  assert_in_delta (e1 + e2) / 2.0, m.func(examples, w), 1e-2, "3"  

  assert_in_delta 3.1622776602, norm(w), 1e-2, "4"
  m2 = LinearRegressionModelL2.new 1.7
  assert_in_delta 9.809225, m2.func(examples, w), 1e-2, "5"
end

test_d0ce9f()

## Question 2.2 (5 Points)

Implement the gradient for the regularized linear regression using the above objective function.

In [154]:
class LinearRegressionModelL2
  def grad examples, w
    g = Hash.new {|h,k| h[k] = 0.0}
    # BEGIN YOUR CODE
    def predict row, w
      x = row["features"]    
      yhat = dot(w, x)
    end
    name = examples[0]["features"].keys
    sum  = Array.new(name.length)
    
    


    adjust w
    
    if (w.keys).empty?
      i = 0
      while i<name.length do
        w.store(name[i],w[0])
        i+=1
      end
      w.delete(0)
    end
    
    norm  = norm w

    i = 0
    while i< w.length do
      
      sum[i] = 0.0
      j = 0
      while j<examples.length
        yhat = predict examples[j], w
        ylabel = examples[j]["label"]
        sum[i] += (yhat -ylabel)*examples[j]["features"][name[i]] 
        j+=1
      end
      sum[i] = (sum[i]/examples.length) + @reg_param * w[name[i]]
      g.store(name[i],sum[i])
      i+=1
    end
    
    
    
    
    #END YOUR CODE
    return g
  end
end

:grad

In [155]:
# w = Hash.new {|h,k| h[k] = 0.0}
# m = LinearRegressionModelL2.new 0.0

# examples = [
#     {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
#     {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
# ]
# dataset = {"data" => examples}
# m.grad(train_set["data"][0,20], w)

In [156]:
### TEST ###
def test_22_0()
  #First, test when there is no regularization term, @reg_param = 0.0
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  # Test on a single example
  expected_g1 = 0.441
  g1 = m.grad(examples[0,1], w)
  puts "g1", g1
  
  assert_in_delta expected_g1, g1["x1"], 1e-2, "1"
  
  # Test on another single example
  expected_g2 = -5.94
  g2 = m.grad(examples[1,1], w)
  puts "g2", g2
  assert_in_delta expected_g2, g2["x1"], 1e-2, "2"  
  
  # Test on both example, should be the average of the two previous gradients
  expected_g3 = (expected_g1 + expected_g2) / 2.0
  g3 = m.grad(examples, w)  
  puts "g3", g3
  assert_in_delta expected_g3, g3["x1"], 1e-2, "3"  
end

test_22_0()

g1
{"x1"=>0.4409999999999999, "x2"=>-0.18899999999999997}
g2
{"x1"=>-5.940000000000001, "x2"=>-2.8600000000000003}
g3
{"x1"=>-2.7495000000000007, "x2"=>-1.5245000000000002}


In [157]:
### TEST ###
def test_ac6e14()
  # Test when the regularization parameter is nonzero
  m2 = LinearRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}

  #Compare this to g3 in the previous test
  g3_with_regularization = m2.grad(examples, w)
  puts "g3_with_regularization", g3_with_regularization  
  assert_in_delta -1.0495, g3_with_regularization["x1"], 1e-2, "1"
  
  #Use a different regularization penalty
  m3 = LinearRegressionModelL2.new 0.3
  g3_with_different_regularization = m3.grad(examples, w)
  puts "g3_with_different_regularization", g3_with_different_regularization  
  assert_in_delta -2.4495, g3_with_different_regularization["x1"], 1e-2, "1"
  
end

test_ac6e14()

g3_with_regularization
{"x1"=>-1.0495000000000008, "x2"=>-6.624499999999999}
g3_with_different_regularization
{"x1"=>-2.449500000000001, "x2"=>-2.4245}


## Question 2.3 (5 Points)

Implement a function that calculates the Root Mean Squared Error (RMSE) for a given dataset for a prediction model and weights.

RMSE is defined as follows:

# $e = \sqrt{\frac{\sum_{i=1}^N{ \left( \hat{y} - y \right) ^ 2 }}{N}}$

where $N$ is the number of examples in the dataset.

Hint: Use the ```mean``` function in the assignment library.

In [158]:
def score_regression_model_rmse(data, weights, model)
  # BEGIN YOUR CODE
  sum = 0.0
  i = 0
  while i< data.length do
    yhat = model.predict data[i],weights
    sum += (yhat - data[i]["label"])**2
    i+=1
  end
  
  return Math.sqrt(sum/data.length)
  
  #END YOUR CODE
end

:score_regression_model_rmse

In [159]:
### TEST ###
def test_23()
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  x = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  e1 = (((0.7 + -3 * -0.3) - 0.97) ** 2)
  e2 = (((-2.7 + -3.0 * -1.3) - -1.0) ** 2)
  
  rmse = Math.sqrt((e1 + e2) / 2.0)
  assert_in_delta rmse, score_regression_model_rmse(x, w, m), 1e-2, "3"  
end

test_23()

## Question 3.1 (10 Points)

Using a small provided dataset, shown below, we will investigate model complexity. First, implement a _polynomial_ feature representation. 

Call the bias feature "1" for this part. For a dataset with two features, $x_1$ and $x_2$, a polynomial representation of degree 0 is as follows:

# $\phi(x, k = 0) = \left( 1 \right)$

degree 1:

# $\phi(x, k = 1) = \left( 1, x_1, x_2 \right)$

degree 2: 

# $\phi(x, k = 2) = \left( 1, x_1, x_2, x_1^2, x_2^2, x_1 x_2  \right)$

and more generally, for degree $k$:

# $\phi(x, k) = \left(1, x_1 \phi(x,k-1), x_2 \phi(x,k-1) \right)$

For your convenience, the function ```poly_features``` emits, for degree $k$, the names of the features to be multiplied. After generating the features, apply ```z_normalize``` to only the newly added features (i.e., not the original features or the bias).

Note: You may notice that the dataset we plan to use only has one feature and therefore the above seems overly complex. Don't worry, we will see this again. ;)

In [160]:
polydata = read_sparse_data_from_csv "polydata-100"
x1 = polydata["data"].collect {|r| r["features"]["x1"]}
x2 = polydata["data"].collect {|r| r["label"]}
puts "Polydata Regression Dataset"
Daru::DataFrame.new({x1: x1, x2: x2})
.plot(type: :scatter, x: :x1, y: :x2) do |plot, diagram|
  plot.x_label "X1"
  plot.y_label "Label"
  plot.legend false
end

Polydata Regression Dataset


#<Nyaplot::Plot:0x000056023ce8e998 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056023ce69710 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2}, :data=>"0fbbab3a-45e0-4073-9941-22b7b714a9b3"}, @xrange=[-2.5225392385889, 2.33601098769754], @yrange=[-3.22266863084871, 4.64672342874836]>], :options=>{:x_label=>"X1", :y_label=>"Label", :legend=>false, :zoom=>true, :width=>700, :xrange=>[-2.5225392385889, 2.33601098769754], :yrange=>[-3.22266863084871, 4.64672342874836]}}>

### Polynomial features

The ```poly_features``` function below returns a set of strings which represents all the possible features for a polynomial feature of degree $k$. In the example above, 

```ruby
poly_features ["x1", "x2"], 3
```

returns all polynomial features up to degree 3, which are as follows:

```["1", "x1", "x2", "1*x1*x1", "1*x1*x2", "1*x2*x2", "1*x1*x1*x1", "1*x1*x1*x2", "1*x1*x2*x2", "1*x2*x2*x2"]```

When provided a ```dataset```, the function ```create_polynomial_features``` you will implement below adds the new features (if they do not already exist) to the ```dataset["features"]``` top-level array. Then, for each example, apply the transformation. This will require manipulating the string and implementing a parser. For example, if the provided feature name is ```"1*x1*x1*x2"```, you will need to calculate $x_1^2 x_2$ as a new feature. Note that each original feature is provided separated by the ```*``` character. Use the ```split``` function for the string.

In [161]:
def poly_features features, degree
  poly_features = ["1"]

  degree.times do |i|
    poly_features += poly_features.flat_map do |x_prev|
      features.reject {|x| x == "1" or x == "bias"}.collect do |x|
        [x, x_prev.split("*")].flatten.sort.join("*")
      end
    end
    poly_features.uniq!
  end
  poly_features.collect {|k| k.gsub /^1\*([^\*]*)$/, '\1'}
end

poly_features ["x1", "x2"], 3

["1", "x1", "x2", "1*x1*x1", "1*x1*x2", "1*x2*x2", "1*x1*x1*x1", "1*x1*x1*x2", "1*x1*x2*x2", "1*x2*x2*x2"]

In [162]:
def create_polynomial_features dataset, degree
  polydataset = dataset.clone
  features = poly_features dataset["features"], degree
  # BEGIN YOUR CODE
  
  already =  polydataset["features"]
  old_data = dataset["data"]
  
  new_features = features-already
  
  if new_features.empty?
    polydataset["features"] = already
  else
    polydataset["features"] = features
  end
  i = 0
  origin = Hash.new

  if new_features.empty?
      polydataset["data"] = old_data
  else
  
        while i<polydataset["data"].length do
          j = 0

              while j<new_features.length do 
                name = new_features[j].split('*',-1)

                data= name.inject(1.0){|sum,x| sum* polydataset["data"][i]["features"][x]}
                polydataset["data"][i]["features"].store(new_features[j],data)

                j+=1
              end

          i+=1
        end
  
  
        polydataset = z_normalize polydataset
        i=0
        while i<polydataset["data"].length do
          k = 0
          while k< already.length do
            polydataset["data"][i]["features"][already[k]] = dataset["data"][i]["features"][already[k]] 
            k+=1
          end

          i+=1
        end
  end
  return polydataset
  #END YOUR CODE
end

:create_polynomial_features

In [163]:
def test_31()
  data = read_sparse_data_from_csv "polydata"
  assert_in_delta 12.8132, data["data"].first["features"]["x1"], 1e-2, "1"
  
  polydata = create_polynomial_features data, 3
  
  xp = polydata["data"].first["features"]
  assert_in_delta 12.8132, xp["x1"], 1e-2, "2: Does not normalize original features"
  assert_in_delta -0.905, xp["1*x1*x1"], 1e-2, "3: Applies normalization to new features"
  assert_in_delta -0.827, xp["1*x1*x1*x1"], 1e-2, "4: Applies normalization to new features"
end

test_31()

## Question 3.2 (5 Points)

Let's fit this dataset with different polynomial degrees. First, let's see how well linear regression fits the training data. 

Implement a training function that, given a training and testing dataset, trains the model using mini-batch SGD and returns the RMSE error value on both training and testing sets.

In [164]:
def train(sgd, obj, w, train_set, test_set, num_epoch = 1, batch_size = 20)
  # BEGIN YOUR CODE


  iters = []
  losses = []

  #Define sgd = StochasticGradientDescent.new(obj, w, lr)


  features=train_set["features"]


  i = 0

 num_epoch.times do
      iters.append(i)
      l = obj.func(train_set["data"][0,batch_size],w)

      losses.append(l)

      sgd.update(train_set["data"][0,batch_size])

      w = sgd.weights

      i+=1
  end
  

  train_rmse = score_regression_model_rmse(train_set["data"][0,batch_size], w, obj)
  test_rmse = score_regression_model_rmse(test_set["data"][0,batch_size], w, obj)
  
  #END YOUR CODE
  return [train_rmse, test_rmse]
end

:train

In [165]:
def test_f472d4()
  data = read_sparse_data_from_csv "polydata"
  polydata = create_polynomial_features data, 1
  x1 = polydata["data"].collect {|r| r["features"]["x1"]}
  x2 = polydata["data"].collect {|r| r["label"]}
  
  w = Hash.new {|h,k| h[k] = 0.0}
  lr = 1e-3
  obj = LinearRegressionModelL2.new 0.0
  sgd = StochasticGradientDescent.new obj, w, lr

  train_set = polydata
  test_set = polydata
  train_rmse, test_rmse = train(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 20)
  assert_true train_rmse < 2, "1"
  assert_true test_rmse < 2, "2"
  assert_true train_rmse > 0, "3"
  assert_true test_rmse > 0, "4"
  assert_in_delta train_rmse, test_rmse, 1e-5, "5"
end

test_f472d4()

## Question 3.3 (2 Points)

Try various degrees to the polynomial regression on this dataset. Make ```answer_452fd8``` return a single integer which is you best guess for the smallest degree which you think fits the data well. Use your best judgment here. The second test plots the training RMSE for a degree from 1 to 16.

In [166]:
def answer_452fd8
  # BEGIN YOUR CODE

  
  
  
  return  4
  
  
  
  
  #END YOUR CODE
  return degree
end

:answer_452fd8

In [167]:
def test_f472d4()
  degree = answer_452fd8()
  data = read_sparse_data_from_csv "polydata-1000"
  polydata = z_normalize create_polynomial_features(data, degree)
   
  w = Hash.new {|h,k| h[k] = 0.0}
  lr = 1e-1
  obj = LinearRegressionModelL2.new 0.0
  sgd = StochasticGradientDescent.new obj, w, lr

  train_set = polydata
  test_set = polydata
  train_rmse, test_rmse = train(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 10)
  
  puts "Weights:", w
  polydata["data"] = polydata["data"].sort_by {|r| r["features"]["x1"]}  
  x1 = polydata["data"].collect {|r| r["features"]["x1"]}
  x2 = polydata["data"].collect {|r| r["label"]}
  yhat = polydata["data"].collect.with_index {|x1, i| obj.predict(x1, w)}
  
  z_plot = Daru::DataFrame.new({x1: x1, x2: x2})
  .plot(type: :scatter, x: :x1, y: :x2) do |plot, diagram|
    plot.x_label "X1"
    plot.y_label "Label"
    plot.legend false
  end

  z_plot.add(:line, x1, yhat).color(:black)
  z_plot.show()  
end

test_f472d4()

Weights:
{"x1"=>0.4300137286830987, "1"=>-0.39694640587476054, "1*x1*x1"=>-0.08261314830583676, "1*x1*x1*x1"=>-0.03304125897578332, "1*x1*x1*x1*x1"=>-0.009150263467552692}


#<Nyaplot::Frame:0x000056023ca650b0 @properties={:panes=>[#<Nyaplot::Plot:0x000056023cdc7618 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056023cadd088 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2}, :data=>"41cf4b5c-aeea-49f0-88a4-9dca6fa0a43a"}, @xrange=[-3.3981146872781, 4.00361923790244], @yrange=[-1.40763254128915, 1.41720360874844]>, #<Nyaplot::Diagram:0x000056023ca655d8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:black}, :data=>"e0e45387-7dc6-486e-baa4-f23cd30991b0"}, @xrange=[-3.3981146872781, 4.00361923790244], @yrange=[-2.258975203900306, 0.22321580790111573]>], :options=>{:x_label=>"X1", :y_label=>"Label", :legend=>false, :zoom=>true, :width=>700, :xrange=>[-3.3981146872781, 4.00361923790244], :yrange=>[-2.258975203900306, 1.41720360874844]}}>], :data=>{"41cf4b5c-aeea-49f0-88a4-9dca6fa0a43a"=>#<Nyaplot::DataFrame:0x000056023caeef68 @name="41cf4b5c-aeea-49f0-88a4-9dca6fa0a43a", @rows=[{:x1=>-3.3981146872781, :x2=>-0.611722153014242}, {:x1=>-3.24143836303274, :x2=>-0.191994602249121}, {:x1=>-2.78228072883387, :x2=>0.992406589438601}, {:x1=>-2.75039586728225, :x2=>1.05446913340098}, {:x1=>-2.74665419119698, :x2=>1.06147959857477}, {:x1=>-2.46437700511943, :x2=>1.39835306982453}, {:x1=>-2.44300327979985, :x2=>1.40687529815457}, {:x1=>-2.41894322460685, :x2=>1.41343114405792}, {:x1=>-2.41287142076604, :x2=>1.41457537770748}, {:x1=>-2.39481386225415, :x2=>1.41676052590908}, {:x1=>-2.38280139198342, :x2=>1.41720360874844}, {:x1=>-2.31818801686942, :x2=>1.40574439730828}, {:x1=>-2.16238676343318, :x2=>1.28408416698451}, {:x1=>-2.06343015184665, :x2=>1.14217126495112}, {:x1=>-2.06131151676102, :x2=>1.13862900722114}, {:x1=>-2.05456094690978, :x2=>1.12720815125042}, {:x1=>-2.02808511689642, :x2=>1.08047858153794}, {:x1=>-2.02478005966374, :x2=>1.07443266654377}, {:x1=>-1.99168009680669, :x2=>1.01137324051041}, {:x1=>-1.95240948360055, :x2=>0.930914075809047}, {:x1=>-1.93266283710784, :x2=>0.888286408179782}, {:x1=>-1.92684345651878, :x2=>0.875460473633285}, {:x1=>-1.91615582681068, :x2=>0.851601762385368}, {:x1=>-1.89746821394705, :x2=>0.808969529860183}, {:x1=>-1.87595523655548, :x2=>0.75851741784037}, {:x1=>-1.85103003404998, :x2=>0.698340569287543}, {:x1=>-1.82147562227814, :x2=>0.624787042235711}, {:x1=>-1.81846805333943, :x2=>0.617177322734159}, {:x1=>-1.79639460682532, :x2=>0.560673203782743}, {:x1=>-1.79263628767266, :x2=>0.550942600597739}, {:x1=>-1.75894055773043, :x2=>0.462400745409513}, {:x1=>-1.74600030752545, :x2=>0.427828680612765}, {:x1=>-1.74514848478333, :x2=>0.425542791686417}, {:x1=>-1.73222717547199, :x2=>0.390723537574267}, {:x1=>-1.72753066025433, :x2=>0.378003267772268}, {:x1=>-1.72607716317567, :x2=>0.374059882359183}, {:x1=>-1.69905321085328, :x2=>0.300215237758907}, {:x1=>-1.68588573611679, :x2=>0.263911186588836}, {:x1=>-1.68559249536125, :x2=>0.263100572700729}, {:x1=>-1.6760512439288, :x2=>0.23667896734448}, {:x1=>-1.66331833299248, :x2=>0.201290355932634}, {:x1=>-1.64099080690236, :x2=>0.13894441294659}, {:x1=>-1.63954388221954, :x2=>0.134893641321292}, {:x1=>-1.63687638463821, :x2=>0.127422981373828}, {:x1=>-1.62930077955886, :x2=>0.106188399573468}, {:x1=>-1.61253408273285, :x2=>0.0591159292086284}, {:x1=>-1.60537459542058, :x2=>0.0389941845588594}, {:x1=>-1.60190068259612, :x2=>0.0292279927318378}, {:x1=>-1.59285733403586, :x2=>0.0038001645303449}, {:x1=>-1.58784335462184, :x2=>-0.0102983892130644}, {:x1=>-1.58473606160155, :x2=>-0.0190349635894179}, {:x1=>-1.57883340533868, :x2=>-0.0356282932302244}, {:x1=>-1.578332888866, :x2=>-0.0370351021678715}, {:x1=>-1.57287217036917, :x2=>-0.0523806256687123}, {:x1=>-1.5713482370769, :x2=>-0.0566620026234729}, {:x1=>-1.57038974221478, :x2=>-0.0593545352328539}, {:x1=>-1.56780193683268, :x2=>-0.0666228118062944}, {:x1=>-1.56057061201395, :x2=>-0.0869225163205443}, {:x1=>-1.54843334635488, :x2=>-0.120949271303153}, {:x1=>-1.54474292577185, :x2=>-0.13128170360486}, {:x1=>-1.54274085587486, :x2=>-0.1368840574923

#<CZTop::Socket::PUB:0x5602340e3da0 last_endpoint="tcp://127.0.0.1:43201">

In [168]:
def test_a6fa85()
  stats = Hash.new {|h,k| h[k] = []}
  
  12.times do |i|
    data = read_sparse_data_from_csv "polydata-1000"
    polydata = z_normalize(create_polynomial_features(data, i))
    obj = LinearRegressionModelL2.new 0.0
    
    w = Hash.new {|h,k| h[k] = 0.0}
    lr = 1e-1
    obj = LinearRegressionModelL2.new 0.0
    sgd = StochasticGradientDescent.new obj, w, lr

    tr_rmse, te_rmse = train(sgd, obj, w, polydata, polydata, num_epoch = 100, batch_size = 10)
    w_norm = norm(w)
    
    stats[:degree] << i
    stats[:train_rmse] << tr_rmse    
    stats[:test_rmse] << te_rmse
    stats[:complexity] << w_norm
  end
  tr_rmse = stats[:train_rmse]
  z_plot = Nyaplot::Plot.new
  z_plot.x_label("Polynomial degree").y_label("Train RMSE")
  z_plot.add(:line, stats[:degree], stats[:train_rmse]).color(:black)
  z_plot.show()  
end
test_a6fa85()

#<Nyaplot::Frame:0x0000560234fd11c8 @properties={:panes=>[#<Nyaplot::Plot:0x0000560234fd3338 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000560234fd1880 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:black}, :data=>"71a2fce2-397c-4e2e-b24b-0c7e9e1d4175"}, @xrange=[0, 11], @yrange=[0.5801734344071452, 0.600069873658112]>], :options=>{:x_label=>"Polynomial degree", :y_label=>"Train RMSE", :zoom=>true, :width=>700, :xrange=>[0, 11], :yrange=>[0.5801734344071452, 0.600069873658112]}}>], :data=>{"71a2fce2-397c-4e2e-b24b-0c7e9e1d4175"=>#<Nyaplot::DataFrame:0x0000560234fd2938 @name="71a2fce2-397c-4e2e-b24b-0c7e9e1d4175", @rows=[{:data0=>0, :data1=>0.600069873658112}, {:data0=>1, :data1=>0.600069873658112}, {:data0=>2, :data1=>0.5861209251805638}, {:data0=>3, :data1=>0.5824621891598809}, {:data0=>4, :data1=>0.5819455396102464}, {:data0=>5, :data1=>0.5801734344071452}, {:data0=>6, :data1=>0.5804396086600726}, {:data0=>7, :data1=>0.5803752511631228}, {:data0=>8, :data1=>0.5804239960745889}, {:data0=>9, :data1=>0.5804324647882203}, {:data0=>10, :data1=>0.580423453836222}, {:data0=>11, :data1=>0.5804217551232491}]>}, :extension=>[]}>

#<CZTop::Socket::PUB:0x5602340e3da0 last_endpoint="tcp://127.0.0.1:43201">

## Question 3.4 (5 points)

In cross validation, we simulate having real test dataset but taking several samples of the data. Cross validation proceeds in $k$ folds deterministically where for 100 examples and 5 folds, we have in the first round, the first 20 examples in test set and the rest in training. For the second rounds, we have examples 20-39 for testing and the remainder for training.

Implement this with a callback ```block``` as follows:

```ruby
def cross_validate dataset, folds, &block
  examples = data["data"]

  folds.times do |fold|
    train_data = dataset.clone
    train_data["data"] = ##CV training examples
    
    test_data = data.clone
    test_data["data"] = ##CV testing examples
    
    ## Call the callback like this:
    yield train_data, test_data, fold
  end
end
```

**Note**: Do not shuffle the data. Preserve the order. Also, make a copy of the dataset so as not to overwrite any datasets or examples. 

In [169]:
def cross_validate dataset, folds, &block
  examples = dataset["data"]
  # BEGIN YOUR CODE
  batch = examples.length/folds
  folds.times do |fold|


      test_data = dataset.clone
      test_data["data"] = test_data["data"][fold*batch,batch]  ##CV training examples

      train_data = dataset.clone
      train_data["data"] = train_data["data"] - test_data["data"]  ##CV testing examples

    ## Call the callback like this:
    yield train_data, test_data, fold
  end
  
  
  #END YOUR CODE
end

:cross_validate

In [170]:
def test_45()
  examples = [
    {"features" => {"x1" => 1}},
    {"features" => {"x2" => 1}},
    {"features" => {"x3" => 1}},
    {"features" => {"x4" => 1}},    
    {"features" => {"x5" => 1}},    
  ]
  dataset = {"data" => examples}
  
  train_sets = []
  test_sets = []
  folds = []
  
  cross_validate dataset, 5 do |train_data, test_data, fold|
    puts "Fold: #{fold}:", "Train Data", train_data, "Test Data:", test_data
    train_sets << train_data
    test_sets << test_data
    folds << fold
    puts ""
  end

  assert_equal [0,1,2,3,4], folds
  assert_true(test_sets[0]["data"][0]["features"].has_key? "x1")
  assert_equal 4, train_sets[0]["data"].size

  assert_true(test_sets[3]["data"][0]["features"].has_key? "x4")
  assert_equal 4, train_sets[3]["data"].size
  assert_true(train_sets[3]["data"][0]["features"].has_key? "x1")

  
end
test_45()

Fold: 0:
Train Data
{"data"=>[{"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x1"=>1}}]}

Fold: 1:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x2"=>1}}]}

Fold: 2:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x3"=>1}}]}

Fold: 3:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x4"=>1}}]}

Fold: 4:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x5"=>1}}]}



## Question 3.5 (6 points)

Let's run 5-fold cross-validation to find the best polynomial degree. If we do this right we should see some difference between the train and test RMSE as we change the degree. Is your choice for the best degree different now?

In [171]:
def test_a6fa85()
  stats = Hash.new {|h,k| h[k] = []}
  
  1.upto(10) do |i|
    data = read_sparse_data_from_csv "polydata-1000"
    polydata = create_polynomial_features(data, i)
    obj = LinearRegressionModelL2.new 0.0
    
    avg_tr_rmse = 0.0
    avg_te_rmse = 0.0
    avg_w_norm = 0.0
    puts "[#{Time.now}] Degree: #{i}"
    cross_validate polydata, 3 do |train_data, test_data, fold|
      w = Hash.new {|h,k| h[k] = 0.0}
      lr = 1e-2
      obj = LinearRegressionModelL2.new 0.0
      sgd = StochasticGradientDescent.new obj, w, lr
      tr_rmse, te_rmse = train(sgd, obj, w, train_data, test_data, num_epoch = 50, batch_size = 20)
      avg_tr_rmse += tr_rmse
      avg_te_rmse += te_rmse
      avg_w_norm += norm(w)
    end
    
    stats[:degree] << i
    stats[:train_rmse] << avg_tr_rmse / 5.0    
    stats[:test_rmse] << avg_te_rmse / 5.0
    stats[:complexity] << avg_w_norm / 5.0
  end
  tr_rmse = stats[:train_rmse]
  z_plot = Nyaplot::Plot.new
  z_plot.x_label("Polynomial degree").y_label("Test RMSE")
  z_plot.add(:line, stats[:degree], stats[:train_rmse]).color(:black)
  z_plot.add(:line, stats[:degree], stats[:test_rmse]).color(:red)  
  z_plot.show()  
end
test_a6fa85()

[2020-11-18 04:18:32 +0000] Degree: 1
[2020-11-18 04:18:32 +0000] Degree: 2
[2020-11-18 04:18:32 +0000] Degree: 3
[2020-11-18 04:18:32 +0000] Degree: 4
[2020-11-18 04:18:33 +0000] Degree: 5
[2020-11-18 04:18:33 +0000] Degree: 6
[2020-11-18 04:18:34 +0000] Degree: 7
[2020-11-18 04:18:34 +0000] Degree: 8
[2020-11-18 04:18:35 +0000] Degree: 9
[2020-11-18 04:18:36 +0000] Degree: 10


#<Nyaplot::Frame:0x000056023cb36700 @properties={:panes=>[#<Nyaplot::Plot:0x000056023cb39478 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056023cb38320 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:black}, :data=>"160f2578-2063-4a06-8aec-7f8a4afe233c"}, @xrange=[1, 10], @yrange=[0.5394012998320631, 0.549305705009345]>, #<Nyaplot::Diagram:0x000056023cb36d68 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:red}, :data=>"cf400b2c-0091-4935-a5e0-77747523d588"}, @xrange=[1, 10], @yrange=[0.5911067203654995, 0.6087515212402084]>], :options=>{:x_label=>"Polynomial degree", :y_label=>"Test RMSE", :zoom=>true, :width=>700, :xrange=>[1, 10], :yrange=>[0.5394012998320631, 0.6087515212402084]}}>], :data=>{"160f2578-2063-4a06-8aec-7f8a4afe233c"=>#<Nyaplot::DataFrame:0x000056023cb38f00 @name="160f2578-2063-4a06-8aec-7f8a4afe233c", @rows=[{:data0=>1, :data1=>0.549305705009345}, {:data0=>2, :data1=>0.5412365815588528}, {:data0=>3, :data1=>0.5424129342927783}, {:data0=>4, :data1=>0.539862398886614}, {:data0=>5, :data1=>0.5401510984183187}, {:data0=>6, :data1=>0.5395253487579404}, {:data0=>7, :data1=>0.5395617031081175}, {:data0=>8, :data1=>0.5394118696245821}, {:data0=>9, :data1=>0.5394365137667919}, {:data0=>10, :data1=>0.5394012998320631}]>, "cf400b2c-0091-4935-a5e0-77747523d588"=>#<Nyaplot::DataFrame:0x000056023cb37a88 @name="cf400b2c-0091-4935-a5e0-77747523d588", @rows=[{:data0=>1, :data1=>0.5911067203654995}, {:data0=>2, :data1=>0.5991399866619697}, {:data0=>3, :data1=>0.5933549171200062}, {:data0=>4, :data1=>0.6039071957673123}, {:data0=>5, :data1=>0.6011880134340273}, {:data0=>6, :data1=>0.607334729507612}, {:data0=>7, :data1=>0.6055427156416335}, {:data0=>8, :data1=>0.6086551440409339}, {:data0=>9, :data1=>0.6071879963345127}, {:data0=>10, :data1=>0.6087515212402084}]>}, :extension=>[]}>

#<CZTop::Socket::PUB:0x5602340e3da0 last_endpoint="tcp://127.0.0.1:43201">

## Question 4.1 (10 Points)

Moving on to classification, implement L2 regularization for Logisitic Regression. This should follow closely what you did in Question 2.X above. 

Use the Log Loss formulation, $\log(1 + \exp(-y\cdot \hat{y}))$ when calculating the objective value.

Map all labels to either 1 or -1.

In [172]:
class LogisticRegressionModelL2
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x = row["features"]    
    1.0 / (1 + Math.exp(-dot(w, x)))
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
  
  def func data, w
    # BEGIN YOUR CODE
    adjust w
    i = 0
    sum  = 0.0

    while i< data.length do
      y=(data[i]["label"])
      yhat=dot(w, data[i]["features"])
      sum += Math.log(Math.exp(-y*yhat)+1.0)
      i+=1
    end

    
    return sum/data.length + 0.5 * @reg_param *(norm(w)**2)
    
    
    
    
    #END YOUR CODE
  end
end

:func

In [173]:
### TEST ###
def test_41_0()
  #Test loss without regularization
  m = LogisticRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  dataset = {"data" => examples}
  e1 = 0.1839007409
  assert_in_delta e1, m.func(examples[0,1], w), 1e-2, "1"
  
  e2 = 1.4632824673
  assert_in_delta e2, m.func(examples[1,1], w), 1e-2, "2"    

  #Loss for 2 examples should be their average
  assert_in_delta (e1 + e2) / 2.0, m.func(examples, w), 1e-2, "3"  
end

test_41_0()

In [174]:
### TEST ###
def test_41_1()
  # Test loss with regularization
  m = LogisticRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  dataset = {"data" => examples}

  assert_in_delta 9.3235916041, m.func(examples, w), 1e-2, "1"
end

test_41_1()

## Question 4.2 (5 Points)

Implement the gradient for L2 regularized Logisitic Regression. As in Assignment 5, use the 0 / 1 version of the loss to simplify the derivation.

In [175]:
# class LogisticRegressionModelL2
#   def grad data, w
#     # BEGIN YOUR CODE

#     g = Hash.new
#     name = data[0]["features"].keys
#     sum  = Array.new(name.length)


#     adjust w
    
#     if (w.keys).empty?
#       i = 0
#       while i<name.length do
#         w.store(name[i],w[0])
#         i+=1
#       end
#       w.delete(0)
#     end
    
#     norm  = norm w

#     i = 0
#     while i< w.length do
      
#       sum[i] = 0.0
#       j = 0
#       while j<data.length
#         yhat =dot  data[j]["features"], w
#         ylabel = data[j]["label"]
#         sum[i] += (1.0/(1.0+Math.exp(ylabel*yhat)))*(-ylabel *data[j]["features"][name[i]] )
# #         sum[i] += (yhat -ylabel)*data[j]["features"][name[i]] 
#         j+=1
#       end
#       sum[i] = (sum[i]/data.length) + @reg_param * w[name[i]]
#       g.store(name[i],sum[i])
#       i+=1
#     end
    
    
#     #END YOUR CODE
#     return g
#   end
# end

In [176]:
class LogisticRegressionModelL2
  def grad data, w
    # BEGIN YOUR CODE
    
    g = Hash.new()
    key_list =[]
    data.each do |e|
        key_list.push(e["features"].keys)
    end
    name = key_list.flatten.uniq{|x| x}

    sum  = Array.new(name.length)


    adjust w
    
    if (w.keys).empty?
      i = 0
      while i<name.length do
        w.store(name[i],w[0])
        i+=1
      end
      w.delete(0)
    end
    
    norm  = norm w

    i = 0
    while i< w.length do
      
      sum = 0.0
      j = 0
      while j<data.length
        yhat =dot  data[j]["features"], w
        ylabel = data[j]["label"]
        if data[j]["features"][name[i]].is_a? Numeric
          sum += (1.0/(1.0+Math.exp(ylabel*yhat)))*(-ylabel *data[j]["features"][name[i]] )
        end
        j+=1
      end
      sum = (sum/data.length) + @reg_param * w[name[i]]
      g.store(name[i],sum)
      i+=1
    end
    
    
    #END YOUR CODE
    return g
  end
end

:grad

In [177]:
### TEST ###
def test_42_0()
  #First without regularization
  m = LogisticRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  g1_1 = -0.1175871304
  assert_in_delta g1_1, m.grad(examples[0,1], w)["x1"], 1e-2, "1"
  
  g2_1 =  -2.0750169154
  assert_in_delta g2_1, m.grad(examples[1,1], w)["x1"], 1e-2, "2"    
  assert_in_delta (g1_1 + g2_1) / 2.0, m.grad(examples, w)["x1"], 1e-2, "3"  
end

test_42_0()

In [178]:
### TEST ###
def test_42_1()
  m = LogisticRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  g1_1 = -0.1175871304
  g2_1 =  -2.0750169154
  assert_in_delta 1.0 * 1.7 + (g1_1 + g2_1) / 2.0, m.grad(examples, w)["x1"], 1e-2, "5"
end

test_42_1()

## Question 4.3 (2 points)

Implement a function that will score your logistic regression model and return an array of pairs of (score, class label).

For the example in the test below, your output should look like this:

```ruby
[
    [0.8320183851339245, 1.0], 
    [0.7685247834990178, 0.0]
]
```

In [179]:
def score_binary_classification_model(data, weights, model)
  # BEGIN YOUR CODE

  scores =Array.new(data.length) { Array.new(2)}
  i = 0
  while i< data.length do
    scores[i][0] = model.predict(data[i],weights)
    scores[i][1] = data[i]["label"]
    i+=1
  end
  
  
  
  #END YOUR CODE
  return scores
end

:score_binary_classification_model

In [180]:
### TEST ###
def test_43()
  m = LogisticRegressionModelL2.new 888.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => 0.0}  
  ]
  
  e1 = 0.8320183851
  e2 = 0.7685247835
  
  scores = score_binary_classification_model(examples, w, m)
  puts scores
  
  assert_in_delta e1, scores[0][0], 1e-2, "1"
  assert_in_delta e2, scores[1][0], 1e-2, "2"  
  assert_in_delta 1.0, scores[0][1], 1e-2, "3"
  assert_in_delta 0.0, scores[1][1], 1e-2, "4"  
end

test_43()

[[0.8320183851339245, 1.0], [0.7685247834990178, 0.0]]


## Question 4.4 (10 Points)

Given the scores defined above, recall that accuracy, true positive, false positive, precision, and recall are defined only for a single threshold $t$. Define confusion matrix given the scores and a threshold. Next, use the confusion matrix to implement all the major metrics. Define each of these measures, given a cut point and assume here that the cut point is defined as $s \ge t$ for score $s$ and threshold $t$. Each function should apply only to the positive class, i.e., when ```label > 0```.

In [181]:
def num_positives scores
  # BEGIN YOUR CODE

  i = 0
  sum = 0
  while i< scores.length do
    if scores[i][1]>0
      sum += 1
    end
    i+=1
  end
  return sum
  
  #END YOUR CODE
end

:num_positives

In [182]:
def num_negatives scores
  # BEGIN YOUR CODE

  return scores.length - num_positives(scores)
  
  #END YOUR CODE
end

:num_negatives

In [183]:
def confusion_matrix(scores, t)
  matrix = Hash.new {|h,predicted_class| h[predicted_class] = Hash.new {|h,true_class| h[true_class] = 0.0}}
  # BEGIN YOUR CODE
  matrix["N"]["N"] = 0
  matrix["N"]["P"] = 0
  matrix["P"]["N"] = 0
  matrix["P"]["P"] = 0
  i = 0
  while i<scores.length do
    if scores[i][1]>0 and scores[i][0]>=t
      matrix["P"]["P"] += 1
    elsif scores[i][1]>0 and scores[i][0]<t
      matrix["N"]["P"] += 1
    elsif scores[i][1]<=0 and scores[i][0]>=t
      matrix["P"]["N"] += 1
    elsif scores[i][1]<=0 and scores[i][0]<t
      matrix["N"]["N"] += 1
    end
    i+=1
  end
  
  #END YOUR CODE
  return matrix
end


:confusion_matrix

In [184]:
def test_44_scores()
  [
    [0.22, 0.0],
    [0.23, 0.0], 
    [0.24, 1.0], 
    [0.25, 0.0],     
    [0.49, 0.0],
    [0.5, 1.0],
    [0.73, 0.0],    
    [0.74, 0.0],
    [0.75, 1.0],    
    [0.76, 1.0],
  ]
end

def test_77fab1
  scores = test_44_scores()
  m1 = confusion_matrix scores, 0.5
  puts "Confusion matrix for scores >= 0.5", m1
  
  assert_equal 4, m1["N"]["N"], "m1[N][N]"
  assert_equal 3, m1["P"]["P"], "m1[P][P]"
  assert_equal 4, num_positives(scores), "num_positives"
  assert_equal 6, num_negatives(scores), "num_negatives"
  
  assert_equal num_negatives(scores), m1["N"]["N"] + m1["P"]["N"], "matrix negatives"
  assert_equal num_positives(scores), m1["N"]["P"] + m1["P"]["P"], "matrix positives"
  
  m2 = confusion_matrix scores, 0.25
  puts "Confusion matrix for scores >= 0.25", m2
  assert_equal 2, m2["N"]["N"], "m2[N][N]"
  assert_equal 3, m2["P"]["P"], "m2[P][P]"
end
test_77fab1

Confusion matrix for scores >= 0.5
{"N"=>{"N"=>4, "P"=>1}, "P"=>{"N"=>2, "P"=>3}}
Confusion matrix for scores >= 0.25
{"N"=>{"N"=>2, "P"=>1}, "P"=>{"N"=>4, "P"=>3}}


In [185]:
def accuracy(matrix, total_pos, total_neg)
  # BEGIN YOUR CODE
  if (total_pos+ total_neg)>0
    sum = (matrix["N"]["N"]+ matrix["P"]["P"]).to_f/(total_pos+ total_neg)
  end
  return sum
  
  #END YOUR CODE
end

:accuracy

In [186]:
### TEST ###
def test_44_0()
  scores = test_44_scores()
  np = num_positives(scores)
  nn = num_negatives(scores)
  m0 = confusion_matrix(scores, 0.0)
  m25 = confusion_matrix(scores, 0.25)
  m50 = confusion_matrix(scores, 0.5)
  m75 = confusion_matrix(scores, 0.75)
  assert_in_delta 0.4, accuracy(m0, np, nn), 1e-3, "t = 0.0"
  assert_in_delta 0.5, accuracy(m25, np, nn), 1e-3, "t = 0.25"
  assert_in_delta 0.7, accuracy(m50, np, nn), 1e-3, "t = 0.5"
  assert_in_delta 0.8, accuracy(m75, np, nn), 1e-3, "t = 0.75"
end

test_44_0()

In [187]:
def true_positive_rate(matrix, total_pos, total_neg)
  # BEGIN YOUR CODE
  if total_pos== 0 and matrix["P"]["P"] == 0
    rate = 1.0
  elsif total_pos>0
    rate =  matrix["P"]["P"].to_f/total_pos
  end
  return  rate
  
  
  #END YOUR CODE
end

:true_positive_rate

In [188]:
### TEST ###
def test_44_1()
  scores = test_44_scores()
  np = num_positives(scores)
  nn = num_negatives(scores)
  m0 = confusion_matrix(scores, 0.0)
  m25 = confusion_matrix(scores, 0.25)
  m50 = confusion_matrix(scores, 0.5)
  m75 = confusion_matrix(scores, 0.75)
  m100 = confusion_matrix(scores, 1.0)  

  assert_in_delta 1.0, true_positive_rate(m0, np, nn), 1e-3, "t = 0.0"
  assert_in_delta 0.75, true_positive_rate(m25, np, nn), 1e-3, "t = 0.25"
  assert_in_delta 0.75, true_positive_rate(m50, np, nn), 1e-3, "t = 0.5"
  assert_in_delta 0.5, true_positive_rate(m75, np, nn), 1e-3, "t = 0.75"
  assert_in_delta 0.0, true_positive_rate(m100, np, nn), 1e-3, "t = 1.0"
end

test_44_1()

In [189]:
def false_positive_rate(matrix, total_pos, total_neg)
  # BEGIN YOUR CODE
  if total_neg== 0 and matrix["P"]["N"] == 0
    rate = 1.0
  elsif total_neg>0
    rate =  matrix["P"]["N"].to_f/total_neg
  end
  return  rate
  
  
  
  #END YOUR CODE
end

:false_positive_rate

In [190]:
### TEST ###
def test_44_2()
  scores = test_44_scores()
  np = num_positives(scores)
  nn = num_negatives(scores)
  m0 = confusion_matrix(scores, 0.0)
  m25 = confusion_matrix(scores, 0.25)
  m50 = confusion_matrix(scores, 0.5)
  m75 = confusion_matrix(scores, 0.75)
  m100 = confusion_matrix(scores, 1.0)  
  
  assert_in_delta 1.0, false_positive_rate(m0, np, nn), 1e-3, "t = 0.0"
  assert_in_delta 0.6666, false_positive_rate(m25, np, nn), 1e-3, "t = 0.25"
  assert_in_delta 0.3333, false_positive_rate(m50, np, nn), 1e-3, "t = 0.5"
  assert_in_delta 0.0, false_positive_rate(m75, np, nn), 1e-3, "t = 0.75"
  assert_in_delta 0.0, false_positive_rate(m100, np, nn), 1e-3, "t = 1.0"
end

test_44_2()

In [191]:
# For precision, set the precision to 1.0 if there are no scored examples
def precision(matrix, total_pos, total_neg)
  # BEGIN YOUR CODE

  p = matrix["P"]["P"] + matrix["P"]["N"]
  if p==0 and matrix["P"]["P"]==0
    rate = 1.0
  elsif p>0
    rate = matrix["P"]["P"].to_f/p
  end
  return rate
  
  #END YOUR CODE
end

:precision

In [192]:
### TEST ###
def test_44_3()
  scores = test_44_scores()
  np = num_positives(scores)
  nn = num_negatives(scores)
  m0 = confusion_matrix(scores, 0.0)
  m25 = confusion_matrix(scores, 0.25)
  m50 = confusion_matrix(scores, 0.5)
  m75 = confusion_matrix(scores, 0.75)
  m100 = confusion_matrix(scores, 1.0)  

  assert_in_delta 0.4, precision(m0, np, nn), 1e-3, "t = 0.0"
  assert_in_delta 0.42857, precision(m25, np, nn), 1e-3, "t = 0.25"
  assert_in_delta 0.6, precision(m50, np, nn), 1e-3, "t = 0.5"
  assert_in_delta 1.0, precision(m75, np, nn), 1e-3, "t = 0.75"
  assert_in_delta 1.0, precision(m100, np, nn), 1e-3, "t = 1.0"
end

test_44_3()

In [193]:
def recall(matrix, total_pos, total_neg)
  # BEGIN YOUR CODE
  if total_pos==0 and matrix["P"]["P"]==0
    rate=1.0
  elsif total_pos>0
    rate = matrix["P"]["P"].to_f/total_pos
  end
  return rate
  
  #END YOUR CODE
end

:recall

In [194]:
### TEST ###
def test_44_4()
  scores = test_44_scores()
  np = num_positives(scores)
  nn = num_negatives(scores)
  m0 = confusion_matrix(scores, 0.0)
  m25 = confusion_matrix(scores, 0.25)
  m50 = confusion_matrix(scores, 0.5)
  m75 = confusion_matrix(scores, 0.75)
  m100 = confusion_matrix(scores, 1.0)  
  
  assert_in_delta 1.0, recall(m0, np, nn), 1e-3, "t = 0.0"
  assert_in_delta 0.75, recall(m25, np, nn), 1e-3, "t = 0.25"
  assert_in_delta 0.75, recall(m50, np, nn), 1e-3, "t = 0.5"
  assert_in_delta 0.5, recall(m75, np, nn), 1e-3, "t = 0.75"
  assert_in_delta 0.0, recall(m100, np, nn), 1e-3, "t = 1.0"
end

test_44_4()

## Question 5.1 (5 Points)

Given an array of pairs of score and class label (0,1), calculate the AUC metric. We will implement this by first implementing the ```roc_curve``` function which computes the points in the ROC curve and at the same time calculates the AUC. If you calculate the confusion matrix incrementally, you may use the metric methods defined above for AUC. 

Recall the definition of AUC as either the area under the ROC curve or the probability of mis-ranking a positive example. Choose one of these methods for the implementation, but we will test that it exectues quickly.


In [195]:
# def roc_curve(scores)
#   fp_rates = [0.0]
#   tp_rates = [0.0]
#   auc = 0.0

#   # BEGIN YOUR CODE
  
#   t_list = (scores.clone).transpose[0].sort!{|x, y| y <=> x}
#   score = (scores.clone).sort!{|x, y| y <=> x}
#   i=0


#   while i< scores.length do
#     matrix = confusion_matrix(score, t_list[i])
#     total_pos = matrix["P"]["P"] + matrix["N"]["P"] 
#     total_neg = matrix["P"]["N"] + matrix["N"]["N"]
#     fp_rates.append(false_positive_rate(matrix, total_pos, total_neg))
#     tp_rates.append(true_positive_rate(matrix, total_pos, total_neg))
#     auc+=(tp_rates[-1]+tp_rates[-2])*(fp_rates[-1]-fp_rates[-2])*0.5

#     i+=1
 
#   end

  
# #   #END YOUR CODE
#   return [fp_rates, tp_rates, auc]
# end

# def calc_auc_only scores
#   fp, tp, auc = roc_curve scores
#   auc
# end


In [196]:
#### this is a debug
  
  def roc_curve(scores)
  fp_rates = [0.0]
  tp_rates = [0.0]
  auc = 0.0

  # BEGIN YOUR CODE

  
  t_list = (scores.clone).transpose[0].sort!{|x, y| y <=> x}
  score = (scores.clone).sort!{|x, y| y <=> x}
  i=0
  matrix = confusion_matrix(score, t_list[i])
  total_pos = matrix["P"]["P"] + matrix["N"]["P"] 
  total_neg = matrix["P"]["N"] + matrix["N"]["N"]
  p = 0.0
  n = 0.0

  score.each do |e|

    if e[1] == 1
      p +=1.0   
      fp_rates.append(fp_rates[-1])      
      tp_rates.append(p/total_pos)
    else
      n +=1.0 
      fp_rates.append(n/total_neg)      
      tp_rates.append(tp_rates[-1])
    end
    auc+=(tp_rates[-1]+tp_rates[-2])*(fp_rates[-1]-fp_rates[-2])*0.5

    i+=1
 
  end
  
  #END YOUR CODE
    return [fp_rates, tp_rates, auc]
  end



  def calc_auc_only scores
    fp, tp, auc = roc_curve scores
    auc
  end

  #END YOUR CODE


:calc_auc_only

In [197]:
def test_51()
  good_model = [[0.9, 1], [0.89, 1], [0.7, 0], [0.8, 1], [0.8, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
  assert_true(calc_auc_only(good_model) > 0.8)
  assert_true(calc_auc_only(good_model) < 1)
  
  srand(777)
  ok_model = Array.new(100) {|i| [100 - i, (rand < (100 - i) / 100.0) ? 1 : 0] }.shuffle
  ok_fp, ok_tp, ok_auc = roc_curve ok_model

  assert_in_delta(0.8631239935587761, ok_auc, 1e-3)
  
  bad_model = Array.new(1000) {|i| [1000 - i, rand < 0.5 ? 1 : 0] }
  bad_auc = calc_auc_only(bad_model)
  assert_in_delta(0.5, bad_auc, 5e-2)
  
  t_51_plot = Nyaplot::Plot.new
  t_51_plot.x_label("False Positive Rate").y_label("True Positive Rate")
  t_51_plot.add(:line, ok_fp, ok_tp).color(:black)
  t_51_plot.show() 

end

test_51()

#<Nyaplot::Frame:0x000056023505d3a8 @properties={:panes=>[#<Nyaplot::Plot:0x000056023506d578 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056023505d948 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:black}, :data=>"a6e30083-413a-4c47-8cb5-a9d097a6a603"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"False Positive Rate", :y_label=>"True Positive Rate", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>], :data=>{"a6e30083-413a-4c47-8cb5-a9d097a6a603"=>#<Nyaplot::DataFrame:0x000056023506cfd8 @name="a6e30083-413a-4c47-8cb5-a9d097a6a603", @rows=[{:data0=>0.0, :data1=>0.0}, {:data0=>0.0, :data1=>0.018518518518518517}, {:data0=>0.0, :data1=>0.037037037037037035}, {:data0=>0.0, :data1=>0.05555555555555555}, {:data0=>0.0, :data1=>0.07407407407407407}, {:data0=>0.0, :data1=>0.09259259259259259}, {:data0=>0.0, :data1=>0.1111111111111111}, {:data0=>0.0, :data1=>0.12962962962962962}, {:data0=>0.0, :data1=>0.14814814814814814}, {:data0=>0.0, :data1=>0.16666666666666666}, {:data0=>0.0, :data1=>0.18518518518518517}, {:data0=>0.0, :data1=>0.2037037037037037}, {:data0=>0.0, :data1=>0.2222222222222222}, {:data0=>0.0, :data1=>0.24074074074074073}, {:data0=>0.0, :data1=>0.25925925925925924}, {:data0=>0.021739130434782608, :data1=>0.25925925925925924}, {:data0=>0.021739130434782608, :data1=>0.2777777777777778}, {:data0=>0.021739130434782608, :data1=>0.2962962962962963}, {:data0=>0.021739130434782608, :data1=>0.3148148148148148}, {:data0=>0.021739130434782608, :data1=>0.3333333333333333}, {:data0=>0.021739130434782608, :data1=>0.35185185185185186}, {:data0=>0.021739130434782608, :data1=>0.37037037037037035}, {:data0=>0.021739130434782608, :data1=>0.3888888888888889}, {:data0=>0.021739130434782608, :data1=>0.4074074074074074}, {:data0=>0.021739130434782608, :data1=>0.42592592592592593}, {:data0=>0.021739130434782608, :data1=>0.4444444444444444}, {:data0=>0.043478260869565216, :data1=>0.4444444444444444}, {:data0=>0.043478260869565216, :data1=>0.46296296296296297}, {:data0=>0.043478260869565216, :data1=>0.48148148148148145}, {:data0=>0.043478260869565216, :data1=>0.5}, {:data0=>0.06521739130434782, :data1=>0.5}, {:data0=>0.06521739130434782, :data1=>0.5185185185185185}, {:data0=>0.06521739130434782, :data1=>0.5370370370370371}, {:data0=>0.06521739130434782, :data1=>0.5555555555555556}, {:data0=>0.06521739130434782, :data1=>0.5740740740740741}, {:data0=>0.06521739130434782, :data1=>0.5925925925925926}, {:data0=>0.06521739130434782, :data1=>0.6111111111111112}, {:data0=>0.06521739130434782, :data1=>0.6296296296296297}, {:data0=>0.08695652173913043, :data1=>0.6296296296296297}, {:data0=>0.10869565217391304, :data1=>0.6296296296296297}, {:data0=>0.10869565217391304, :data1=>0.6481481481481481}, {:data0=>0.10869565217391304, :data1=>0.6666666666666666}, {:data0=>0.13043478260869565, :data1=>0.6666666666666666}, {:data0=>0.15217391304347827, :data1=>0.6666666666666666}, {:data0=>0.15217391304347827, :data1=>0.6851851851851852}, {:data0=>0.17391304347826086, :data1=>0.6851851851851852}, {:data0=>0.17391304347826086, :data1=>0.7037037037037037}, {:data0=>0.1956521739130435, :data1=>0.7037037037037037}, {:data0=>0.21739130434782608, :data1=>0.7037037037037037}, {:data0=>0.21739130434782608, :data1=>0.7222222222222222}, {:data0=>0.2391304347826087, :data1=>0.7222222222222222}, {:data0=>0.2391304347826087, :data1=>0.7407407407407407}, {:data0=>0.2608695652173913, :data1=>0.7407407407407407}, {:data0=>0.2608695652173913, :data1=>0.7592592592592593}, {:data0=>0.2608695652173913, :data1=>0.7777777777777778}, {:data0=>0.2608695652173913, :data1=>0.7962962962962963}, {:data0=>0.2608695652173913, :data1=>0.8148148148148148}, {:data0=>0.2608695652173913, :data1=>0.8333333333333334}, {:data0=>0.2826086956521739, :data1=>0.8333333333333334}, {:data0=>0.2826086956521739, :data1=>0.8518518518518519}, {:data0=>0.2826086956521739, :data1=>0.8703703703703703}, {:data0=>0.30434782608695654, :data1=>0.87037037

#<CZTop::Socket::PUB:0x5602340e3da0 last_endpoint="tcp://127.0.0.1:43201">

In [198]:
#Tests whether model can handle a large number of examples
# In the professor's implementation, this takes less than 1 minute
def test_759930()
  srand(777)
  2.upto(6) do |d|
    n = 10 ** d
    bad_model = Array.new(n) {|i| [n - i, rand < 0.5 ? 1 : 0] }.shuffle
    bad_auc = calc_auc_only(bad_model)
    assert_in_delta(0.5, bad_auc, 5e-2, n)
    puts "At: #{Time.now}, #{n} points. AUC = #{bad_auc}"
    
    ok_model = Array.new(n) {|i| [n - i, (rand < (n - i) / n.to_f) ? 1 : 0] }.shuffle
    ok_auc = calc_auc_only(ok_model)
    assert_in_delta(0.83, ok_auc, 0.05)
    puts "At: #{Time.now}, #{n} points. AUC = #{ok_auc}"    
  end
  return
end

test_759930()

At: 2020-11-18 04:18:37 +0000, 100 points. AUC = 0.503001200480192
At: 2020-11-18 04:18:37 +0000, 100 points. AUC = 0.8683473389355744
At: 2020-11-18 04:18:37 +0000, 1000 points. AUC = 0.48840532146516374
At: 2020-11-18 04:18:37 +0000, 1000 points. AUC = 0.8395280000000002
At: 2020-11-18 04:18:37 +0000, 10000 points. AUC = 0.5016753481086853
At: 2020-11-18 04:18:37 +0000, 10000 points. AUC = 0.8348011938195139
At: 2020-11-18 04:18:38 +0000, 100000 points. AUC = 0.49883707973997193
At: 2020-11-18 04:18:39 +0000, 100000 points. AUC = 0.8325222069079209
At: 2020-11-18 04:18:50 +0000, 1000000 points. AUC = 0.49956284583976207
At: 2020-11-18 04:19:01 +0000, 1000000 points. AUC = 0.8331191662603503


## Question 5.2 (5 Points)

The following dataset has _irrelevant features_. Find them and use regularization to control them. 

Implement a training method that trains a logistic regression model and returns training and testing AUC values. This follows closely question 3.2 above. Next, fill in the driver code that trains the model for each regularization value and populates an array of training AUC, testing AUC, and weight vector norm values.

Hint: The weights for regularization parameter are displayed.

In [199]:
def train_logistic_regression(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 20)
  # BEGIN YOUR CODE

    iters = []
    losses = []
    features=train_set["features"]
    i = 0
   num_epoch.times do
        iters.append(i)
        l = obj.func(train_set["data"][0,batch_size],w)

        losses.append(l)

        sgd.update(train_set["data"][0,batch_size])

        w = sgd.weights

        i+=1
    end


  train_score = score_binary_classification_model(train_set["data"], w, obj)
  train_auc = calc_auc_only(train_score)

  test_score = score_binary_classification_model(test_set["data"], w, obj)
  test_auc = calc_auc_only(test_score)
  #END YOUR CODE
  return [train_auc, test_auc]
end


:train_logistic_regression

In [200]:
def test_logistic_regularizers(corner)
  stats = Hash.new {|h,k| h[k] = Array.new}
  [0.0, 0.5, 1.0, 5.0, 7.0, 10.0, 12.0, 15.0, 20.0, 25.0, 30.0, 35.0].each do |reg|
    tr_aucs = []
    te_aucs = []
    w_norms = []

    cross_validate corner, 5 do |tr, te, fold|
      # BEGIN YOUR CODE
     
      lr = 1e-1
      w = Hash.new {|h,k| h[k] =0.0}
      obj = LogisticRegressionModelL2.new(reg)
      sgd = StochasticGradientDescent.new(obj, w, lr)

      a = train_logistic_regression(sgd, obj, w, tr, te, num_epoch = 100, batch_size = 20)

      tr_aucs.append(a[0])
      te_aucs.append(a[1])
      w_norms.append(norm w)

      
      
      #END YOUR CODE
      puts "Reg: #{reg}", "Weights", w if fold == 0
    end
    stats[:reg] << reg
    stats[:tr_aucs] << mean(tr_aucs)
    stats[:tr_aucs_std] << stdev(tr_aucs)
    stats[:w_norms] << mean(w_norms)
    stats[:te_aucs] << mean(te_aucs)    
    stats[:te_aucs_std] << stdev(te_aucs)        
  end
  
  return stats
end

:test_logistic_regularizers

In [201]:
def test_52()
  corner = z_normalize(read_sparse_data_from_csv("corner"))
  corner["data"].first
  
  stats = test_logistic_regularizers(corner)
  assert_true(stats[:tr_aucs].all? {|a| a > 0.7 and a < 1.0}, "1")
  assert_true(stats[:te_aucs].all? {|a| a > 0.7 and a < 1.0}, "2")
  assert_true(stats[:w_norms][0] > stats[:w_norms][6], "3")
  assert_true(stats[:w_norms][6] > stats[:w_norms].last, "4")    
  [Daru::DataFrame.new(stats), stats]
end

df, t_52_stats = test_52()
df

Reg: 0.0
Weights
{"x1"=>-0.02375866336007571, "x2"=>-0.11676290665253265, "x3"=>-0.210418197727949, "x4"=>0.07117114944251744, "x5"=>-0.08455935357116226, "x6"=>-0.056636891668571075, "1"=>-0.6586105732454504}
Reg: 0.5
Weights
{"x1"=>-0.018440258552157997, "x2"=>-0.07741388432373186, "x3"=>-0.14196714416799514, "x4"=>0.04643436515649964, "x5"=>-0.06222954386828057, "x6"=>-0.04329157745857091, "1"=>-0.44490935113651126}
Reg: 1.0
Weights
{"x1"=>-0.01497420342506096, "x2"=>-0.0548319316107266, "x3"=>-0.10235024649230903, "x4"=>0.03222936537601557, "x5"=>-0.048581327110121836, "x6"=>-0.03490599911725995, "1"=>-0.32054890975539296}
Reg: 5.0
Weights
{"x1"=>-0.005596563383435679, "x2"=>-0.013709659283225114, "x3"=>-0.027370019435746074, "x4"=>0.00719464140632493, "x5"=>-0.01676708869819052, "x6"=>-0.01321879629273838, "1"=>-0.0850945530080072}
Reg: 7.0
Weights
{"x1"=>-0.0042134357027029945, "x2"=>-0.009867443049658406, "x3"=>-0.019880875963932558, "x4"=>0.005061614545096915, "x5"=>-0.01257023

,reg,tr_aucs,tr_aucs_std,w_norms,te_aucs,te_aucs_std
0,0.0,0.893434491901073,0.01368179705377354,0.7733748779259466,0.9140002241266789,0.04300713804110452
1,0.5,0.8924881319803821,0.01002964542533726,0.5262000203631721,0.9150692752690548,0.04182695647963343
2,1.0,0.8902606294005835,0.007522310730732953,0.3824021006550716,0.9140436342434135,0.041071696793941244
3,5.0,0.8810214192921764,0.018234886034951794,0.1053284372430818,0.9093226208019566,0.03712223882574103
4,7.0,0.8793861648976963,0.02160596520948303,0.07682048945967787,0.9085591976965646,0.0371905515226509
5,10.0,0.8784093541379165,0.02415449936459205,0.0546457951735972,0.9078867233660592,0.036842966027864084
6,12.0,0.8776634356835242,0.02549978756068069,0.04582859952529195,0.907771991775338,0.037190853724201244
7,15.0,0.8770062809157786,0.026856934815955545,0.03689894195878553,0.9079200302653454,0.036972922632489184
8,20.0,0.8764940439228681,0.027885018747266063,0.027854096950097367,0.907771991775338,0.037190853724201244
9,25.0,0.8762269079352046,0.028691350274705454,0.02237073685053468,0.9079200302653454,0.036972922632489184


### Interpretation

Running 5-fold cross validation for several parameter combinations, we can find the best regularization parameter based on the average of the test sets obtained during cross validation. We call them test sets, but they are subsets of the original dataset--just not provided to the trainer. The plot below will help you find the best value. 

In [202]:
t_52_plot = Nyaplot::Plot.new
t_52_plot.x_label("Regularization Parameter").y_label("Average CV AUC")
t_52_plot.add(:line, t_52_stats[:reg], t_52_stats[:te_aucs]).color(:red)
t_52_plot.show() 

#<Nyaplot::Frame:0x000056023f6b0750 @properties={:panes=>[#<Nyaplot::Plot:0x000056023f6b1fb0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000056023f6b0c78 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:red}, :data=>"8a557c2c-2e97-4e6d-905d-397555e9727d"}, @xrange=[0.0, 35.0], @yrange=[0.907771991775338, 0.9150692752690548]>], :options=>{:x_label=>"Regularization Parameter", :y_label=>"Average CV AUC", :zoom=>true, :width=>700, :xrange=>[0.0, 35.0], :yrange=>[0.907771991775338, 0.9150692752690548]}}>], :data=>{"8a557c2c-2e97-4e6d-905d-397555e9727d"=>#<Nyaplot::DataFrame:0x000056023f6b1a38 @name="8a557c2c-2e97-4e6d-905d-397555e9727d", @rows=[{:data0=>0.0, :data1=>0.9140002241266789}, {:data0=>0.5, :data1=>0.9150692752690548}, {:data0=>1.0, :data1=>0.9140436342434135}, {:data0=>5.0, :data1=>0.9093226208019566}, {:data0=>7.0, :data1=>0.9085591976965646}, {:data0=>10.0, :data1=>0.9078867233660592}, {:data0=>12.0, :data1=>0.907771991775338}, {:data0=>15.0, :data1=>0.9079200302653454}, {:data0=>20.0, :data1=>0.907771991775338}, {:data0=>25.0, :data1=>0.9079200302653454}, {:data0=>30.0, :data1=>0.9079200302653454}, {:data0=>35.0, :data1=>0.9079200302653454}]>}, :extension=>[]}>

#<CZTop::Socket::PUB:0x5602340e3da0 last_endpoint="tcp://127.0.0.1:43201">

## Question 5.3 (2 Points)

Make the function below return an array of feature names you think are irrelevant. If you think features ```"a", "b"``` are irrelevant, write your function below as follows:

```ruby
def guess_irrelevant_features()
  answer = ["a", "b"]
  return answer
end
```

**Hint**: What does L2 regularization do for irrelevant features?

In [203]:
def guess_irrelevant_features()
  # BEGIN YOUR CODE

  answer=["x4","x5","x6"]
  
  #END YOUR CODE
  return answer
end

:guess_irrelevant_features

In [204]:
corner = read_sparse_data_from_csv("corner")

t53_answer = guess_irrelevant_features()
assert_true(t53_answer.is_a?(Array))
assert_false(t53_answer.empty?)
assert_false((corner["features"] & t53_answer).empty?)

### Hidden Test ###


## Question 5.4 (3 Points)

Based on the plot above for different regularization parameters, which has the best AUC?  If your answer is, say 1.2345, write the following: 

```ruby
def pick_best_regularizer()
  answer = 1.2345
  return answer
end
```

In [205]:
def pick_best_regularizer()
  # BEGIN YOUR CODE
  answer = 0.5
  #END YOUR CODE
  return answer
end

:pick_best_regularizer

In [206]:
### Hidden Test ###